In [1]:
import sys
sys.path.append('/home/jovyan/work/')
sys.path.append('/home/jovyan/work/src')
sys.path.append('/home/jovyan/work/src/data')


In [75]:
from data import create_input_data
df_jan = create_input_data.read_data(csv_file_name="../data/processed/lookback35_anon_jan1_feb1.csv", parse_dates_col=[4,5,6])
df_feb = create_input_data.read_data(csv_file_name="../data/processed/lookback35_anon_feb1_aug1.csv", parse_dates_col=[4,5,6])

In [91]:
df_jan.info()
df_feb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33497 entries, 0 to 33496
Data columns (total 52 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   jobid             33497 non-null  int64         
 1   user              33497 non-null  object        
 2   account           33497 non-null  object        
 3   state             33497 non-null  object        
 4   submit            33497 non-null  datetime64[ns]
 5   start             33497 non-null  datetime64[ns]
 6   end               33497 non-null  datetime64[ns]
 7   reqcpus           33497 non-null  int64         
 8   nnodes            33497 non-null  int64         
 9   max_minutes       33497 non-null  int64         
 10  queue_minutes     33497 non-null  int64         
 11  backlog_minutes   33497 non-null  int64         
 12  backlog_num_jobs  33497 non-null  int64         
 13  running_num_jobs  33497 non-null  int64         
 14  running_minutes   3349

In [92]:
import numpy as np 
# df = df_jan
df = df_feb

# How many jobs sat in queue for 0 minutes?
nbr_jobs = len(df)
nbr_zero_queue_min_jobs = len(df[df.queue_minutes == 0])
small_minutes_threshhold = 5
nbr_small_queue_min_jobs = len(df[df.queue_minutes <= small_minutes_threshhold])
print("Initial total number of jobs: ", nbr_jobs)
print("Jobs with 0 queue minutes: ", nbr_zero_queue_min_jobs)
print(f"Jobs with queue minutes leq {small_minutes_threshhold}: ", nbr_small_queue_min_jobs)
print("Jobs with queue minutes: ", nbr_zero_queue_min_jobs)
print("Percentage of jobs with 0 queue minutes: ", float(nbr_zero_queue_min_jobs)/nbr_jobs)

# Optionally, drop the rows (jobs) that sat in queue for 0 minutes 
DROP_ZERO_QUEUE_MINUTES = True
if DROP_ZERO_QUEUE_MINUTES:
    df = df.drop(df[df.queue_minutes == 0].index)
    min_queue_minutes = df['queue_minutes'].min()
    print("After dropping zero minute jobs, shortest queue time is: ", min_queue_minutes)

print("Jobs with 0 queue minutes: ", nbr_zero_queue_min_jobs)# How many jobs sat in queue for > more than x days?
NBR_DAYS_HIGH = 2
nbr_minutes_high = NBR_DAYS_HIGH * 24 * 60 
nbr_high_queue_min_jobs = len(df[df.queue_minutes >= nbr_minutes_high])
print("HIGH number of days threshold: ", NBR_DAYS_HIGH, " (", nbr_minutes_high, " minutes)")
print("Jobs with HIGH queue minutes: ", nbr_high_queue_min_jobs)
print("Percentage of jobs with HIGH queue minutes: ", float(nbr_high_queue_min_jobs)/nbr_jobs)

# Optionally, drop the rows (jobs) that sat in queue for 0 minutes 
DROP_HIGH_QUEUE_MINUTES = True
if DROP_HIGH_QUEUE_MINUTES:
    df = df.drop(df[df.queue_minutes >= nbr_minutes_high].index)

print("Final total number of jobs: ", len(df))

# Add a new column, queue_minutes_bin, which is the bin of the 
bin_threshhold = 10
bin_size_factor = 6 # implies bins of size 60 min
bin_size = bin_threshhold * bin_size_factor
nbr_bins = 5

df['queue_minutes_bin'] = df['queue_minutes'] / bin_size
df = df.astype({'queue_minutes_bin': 'int'})
df['queue_minutes_bin'].unique()
# for jobs in a bin number larger than the number of bins -1 (bins are 0-indexed), just 
# put them in the largest bin. 
df['queue_minutes_bin'] = np.where(df['queue_minutes_bin'] > (nbr_bins - 1), (nbr_bins - 1), df['queue_minutes_bin'])
# print the final bin counts
df['queue_minutes_bin'].value_counts()

Initial total number of jobs:  323216
Jobs with 0 queue minutes:  198478
Jobs with queue minutes leq 5:  220475
Jobs with queue minutes:  198478
Percentage of jobs with 0 queue minutes:  0.6140723231523192
After dropping zero minute jobs, shortest queue time is:  1
Jobs with 0 queue minutes:  198478
HIGH number of days threshold:  2  ( 2880  minutes)
Jobs with HIGH queue minutes:  1821
Percentage of jobs with HIGH queue minutes:  0.005634003267164992
Final total number of jobs:  122917


queue_minutes_bin
0    60506
4    30285
1    15776
2     9578
3     6772
Name: count, dtype: int64

In [86]:
import numpy as np 
def split_df_current_future(df, cutoff_fraction=0.75, cutoff_datetime=None):
    """
    Split a jobs dataframe into a current and future set. 
      * cutoff_fraction (float): the fraction to use for current.
      * cutoff_datetime, if supplied, should be the date_time to use as the cutoff point.
        When supplied, cutoff_fraction is ignored.
 
    Returns two dataframes, current and future
    """
    # if cutoff_datetime is provided, just use that 
    if cutoff_datetime: 
        current = pd[pd['submit'] <= cutoff_dateime]
        future = pd[pd['submit'] > cutoff_dateime]
        return current, future 
    # otherwise, we are using cutoff_fraction. 
    if not 0 < cutoff_fraction < 1:
        print("Invalid cutoff_fraction; values should be between 0 and 1.")
        return None, None 
    # the following uses np.quantile to split the df on the submit column using the cutoff_fraction
    current = df[df['submit']<=np.quantile(df['submit'], cutoff_fraction )]
    future = df[df['submit']>np.quantile(df['submit'], cutoff_fraction )]
    return current, future

In [88]:
current, future = split_df_current_future(df)
print("Current: ", len(current), " Future: ", len(future))
print("Max current: ", current["submit"].max(), " Min future: ", future['submit'].min())

Current:  9221  Future:  3073
Max current:  2022-01-27 12:38:39  Min future:  2022-01-27 12:38:41


In [93]:
# These are the columns of the independent and dependent vars, repsectively:
X_cols = ['nnodes', 
          'max_minutes', 
          'backlog_minutes', 
          'backlog_num_jobs', 
          'running_num_jobs', 
          'running_minutes',
         ]
y_col = 'queue_minutes_bin'

# whether to train on the full dataset (after splitting) or to use the "current"
TRAIN_ON_FULL = True 

# Train-test split --------
from sklearn.model_selection import train_test_split
# Create independent and dependent vars
X = df[X_cols]
X_current = current[X_cols]
X_future = future[X_cols]
y = df[y_col]
y_current = current[y_col]
y_future = future[y_col]

if TRAIN_ON_FULL:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)
else:
    X_train, X_test, y_train, y_test = train_test_split(X_current, y_current, test_size=0.2, stratify=y, random_state=1)

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression(random_state=1, max_iter=1000)
p = pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('lr', lr),
])

# fit the pipeline 
p.fit(X_train, y_train)
model = p 
# or, fit the LG model by hand with no standardization 
# model = lr.fit(X_train, y_train)

# print the report
print(f"Performance on TEST\n*******************\n{classification_report(y_test, model.predict(X_test))}")
print(f"Performance on TRAIN\n********************\n{classification_report(y_train, model.predict(X_train))}")

Performance on TEST
*******************
              precision    recall  f1-score   support

           0       0.49      0.81      0.61       991
           1       0.17      0.03      0.06       365
           2       0.27      0.09      0.13       268
           3       0.27      0.01      0.03       200
           4       0.50      0.53      0.52       635

    accuracy                           0.48      2459
   macro avg       0.34      0.29      0.27      2459
weighted avg       0.41      0.48      0.40      2459

Performance on TRAIN
********************
              precision    recall  f1-score   support

           0       0.48      0.78      0.59      3966
           1       0.18      0.04      0.06      1459
           2       0.24      0.07      0.11      1071
           3       0.20      0.01      0.02       798
           4       0.49      0.51      0.50      2541

    accuracy                           0.46      9835
   macro avg       0.32      0.28      0.26      

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler 
from sklearn.neighbors import KNeighborsClassifier

p = pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=3)),
])

model = p.fit(X_train, y_train)

# p = pipeline = Pipeline([
#     ('scale', StandardScaler()),
#     ('knn', KNeighborsClassifier()),
# ])
# param_grid = {
#     "knn__n_neighbors": np.arange(1, 100)
# }
# search = GridSearchCV(p, param_grid, n_jobs=8, refit=True)
# search.fit(X_train, y_train)
# print(f"Score with best parameters: {search.best_score_}")
# print(search.best_params_)
# model = search.best_estimator_

print(f"Performance on TEST\n*******************\n{classification_report(y_test, model.predict(X_test))}")
print(f"Performance on TRAIN\n********************\n{classification_report(y_train, model.predict(X_train))}")

Performance on TEST
*******************
              precision    recall  f1-score   support

           0       0.90      0.92      0.91       991
           1       0.82      0.78      0.80       365
           2       0.82      0.84      0.83       268
           3       0.86      0.78      0.82       200
           4       0.90      0.91      0.91       635

    accuracy                           0.88      2459
   macro avg       0.86      0.85      0.85      2459
weighted avg       0.88      0.88      0.88      2459

Performance on TRAIN
********************
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3966
           1       0.92      0.88      0.90      1459
           2       0.93      0.90      0.91      1071
           3       0.93      0.88      0.91       798
           4       0.95      0.96      0.96      2541

    accuracy                           0.94      9835
   macro avg       0.93      0.92      0.92      

In [58]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

model = gnb.fit(X_train, y_train)

print(f"Performance on TEST\n*******************\n{classification_report(y_test, model.predict(X_test))}")
print(f"Performance on TRAIN\n********************\n{classification_report(y_train, model.predict(X_train))}")

Performance on TEST
*******************
              precision    recall  f1-score   support

           0       0.60      0.50      0.54       991
           1       0.30      0.10      0.15       365
           2       0.20      0.61      0.30       268
           3       0.00      0.00      0.00       200
           4       0.55      0.59      0.57       635

    accuracy                           0.43      2459
   macro avg       0.33      0.36      0.31      2459
weighted avg       0.45      0.43      0.42      2459

Performance on TRAIN
********************
              precision    recall  f1-score   support

           0       0.57      0.48      0.52      3966
           1       0.26      0.08      0.13      1459
           2       0.19      0.61      0.29      1071
           3       0.00      0.00      0.00       798
           4       0.54      0.57      0.56      2541

    accuracy                           0.42      9835
   macro avg       0.31      0.35      0.30      

/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.11/site-packages/sklearn/metrics/_classif

In [72]:
from sklearn.svm import SVC

p = pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC()),
])
param_grid = {
    
    # kernel functions to try
    "svc__kernel": [
        # 'poly', 
        'rbf', 
        # 'sigmoid'
    ],
    
    # polynomial dregrees to try, for kernel==poly
    # "svc__degree": np.arange(start=2, stop=5),
    
    # regularization param ; bigger C tends to overfit
    # "svc__C": [0.1, 1, 10, 100, 1000],
    # "svc__C": np.arange(start=1000, stop=2000, step=100),
    "svc__C": np.arange(start=1500, stop=2000, step=100),
    
    # kernel coeff; 
    "svc__gamma": [1, 0.1, 0.01,]# 0.001, 0.0001], # ["scale", "auto"]
}

search = GridSearchCV(p, param_grid, n_jobs=8, refit=True, verbose=2) #verbose for progress logging
search.fit(X_train, y_train)
print(f"Score with best parameters: {search.best_score_}")
print(search.best_params_)
model = search.best_estimator_

print(f"Performance on TEST\n*******************\n{classification_report(y_test, model.predict(X_test))}")
print(f"Performance on TRAIN\n********************\n{classification_report(y_train, model.predict(X_train))}")

Fitting 5 folds for each of 50 candidates, totalling 250 fits
Score with best parameters: 0.8638535841382817
{'svc__C': 1700, 'svc__gamma': 1, 'svc__kernel': 'rbf'}
Performance on TEST
*******************
              precision    recall  f1-score   support

           0       0.89      0.92      0.90       991
           1       0.78      0.76      0.77       365
           2       0.84      0.80      0.82       268
           3       0.80      0.83      0.82       200
           4       0.93      0.90      0.91       635

    accuracy                           0.87      2459
   macro avg       0.85      0.84      0.84      2459
weighted avg       0.87      0.87      0.87      2459

Performance on TRAIN
********************
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      3966
           1       0.90      0.87      0.89      1459
           2       0.92      0.88      0.90      1071
           3       0.92      0.94      0.93    

In [94]:
p = pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('svc', SVC()),
])
param_grid = {
    
    # kernel functions to try
    "svc__kernel": [
        # 'poly', 
        'rbf', 
        # 'sigmoid'
    ],
    
    # polynomial dregrees to try, for kernel==poly
    # "svc__degree": np.arange(start=2, stop=5),
    
    # regularization param ; bigger C tends to overfit
    # "svc__C": [0.1, 1, 10, 100, 1000],
    # "svc__C": np.arange(start=1000, stop=2000, step=100),
    "svc__C": np.arange(start=1500, stop=2000, step=100),
    
    # kernel coeff; 
    "svc__gamma": [1, 0.1, 0.01,]# 0.001, 0.0001], # ["scale", "auto"]
}

search = GridSearchCV(p, param_grid, n_jobs=8, refit=True, verbose=2) #verbose for progress logging
search.fit(X_train, y_train)
print(f"Score with best parameters: {search.best_score_}")
print(search.best_params_)
model = search.best_estimator_

print(f"Performance on TEST\n*******************\n{classification_report(y_test, model.predict(X_test))}")
print(f"Performance on TRAIN\n********************\n{classification_report(y_train, model.predict(X_train))}")

Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV] END .........svc__C=1500, svc__gamma=1, svc__kernel=rbf; total time=73.1min
[CV] END ......svc__C=1500, svc__gamma=0.01, svc__kernel=rbf; total time=25.6min
[CV] END .......svc__C=1600, svc__gamma=0.1, svc__kernel=rbf; total time=39.9min
[CV] END ......svc__C=1600, svc__gamma=0.01, svc__kernel=rbf; total time=21.2min
[CV] END .........svc__C=1700, svc__gamma=1, svc__kernel=rbf; total time=66.2min
[CV] END ......svc__C=1700, svc__gamma=0.01, svc__kernel=rbf; total time=21.6min
[CV] END .........svc__C=1800, svc__gamma=1, svc__kernel=rbf; total time=67.2min
[CV] END ......svc__C=1800, svc__gamma=0.01, svc__kernel=rbf; total time=21.9min
[CV] END .......svc__C=1900, svc__gamma=0.1, svc__kernel=rbf; total time=42.4min
[CV] END ......svc__C=1900, svc__gamma=0.01, svc__kernel=rbf; total time=22.2min
[CV] END .........svc__C=1500, svc__gamma=1, svc__kernel=rbf; total time=73.4min
[CV] END .........svc__C=1600, svc__gamma=1, svc

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))